In [ ]:

import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer

df = pd.read_csv('Housing.csv')

binary_cols = ['mainroad', 'guestroom', 'basement', 'hotwaterheating', 'airconditioning', 'prefarea']
for col in binary_cols:
    df[col] = df[col].map({'yes': 1, 'no': 0})

X = df.drop('price', axis=1)
y = df['price']

numerical_features = ['area', 'bedrooms', 'bathrooms', 'stories', 'parking']
categorical_features = ['furnishingstatus']

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),
        ('cat', OneHotEncoder(), categorical_features)
    ],
    remainder='passthrough'
)

X_processed = preprocessor.fit_transform(X)

y_reshaped = y.values.reshape(-1, 1)
y_scaler = StandardScaler()
y_scaled = y_scaler.fit_transform(y_reshaped)

X_train, X_test, y_train, y_test = train_test_split(X_processed, y_scaled, test_size=0.2, random_state=42)

model = keras.Sequential([
    layers.Dense(128, activation='relu', input_shape=[X_train.shape[1]]),

    layers.Dense(64, activation='relu'),

    layers.Dense(32, activation='relu'),

    layers.Dense(1)
])

model.compile(optimizer='adam',
              loss='mean_squared_error')

print("--- Starting Model Training ---")

history = model.fit(
    X_train, y_train,
    epochs=100,
    validation_split=0.2,
    verbose=0
)

print("--- Model Training Finished ---")

loss = model.evaluate(X_test, y_test, verbose=0)
print(f"\nModel's Final Mean Squared Error on Test Data: {loss:.4f}")

predictions_scaled = model.predict(X_test)

predictions = y_scaler.inverse_transform(predictions_scaled).flatten()
actual_prices = y_scaler.inverse_transform(y_test).flatten()

comparison_df = pd.DataFrame({
    'Actual Price': actual_prices,
    'Predicted Price': predictions
})
comparison_df['Difference'] = comparison_df['Actual Price'] - comparison_df['Predicted Price']

print("\n Prediction Comparison (First 5 Result) ")
print(comparison_df.head())

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


--- Starting Model Training ---
--- Model Training Finished ---

Model's Final Mean Squared Error on Test Data: 0.5966
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step

--- Prediction Comparison (First 5 Results) ---
   Actual Price  Predicted Price  Difference
0     4060000.0       5649185.50 -1589185.50
1     6650000.0       7988094.50 -1338094.50
2     3710000.0       3715624.00    -5624.00
3     6440000.0       4681738.00  1758262.00
4     2800000.0       3566060.25  -766060.25
